In [1]:
#!pip install scikit-image

In [2]:
!pip install scikit-image

import skimage


import findspark
findspark.init()

import pyspark
from pyspark.ml.feature import PCA, VectorAssembler
from pyspark.mllib.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark PCA example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


Defaulting to user installation because normal site-packages is not writeable
distutils: /home/hadoop/.local/lib/python3.9/site-packages
sysconfig: /home/hadoop/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 03:27:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/05 03:27:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df =  spark.read.format('csv').\
                       options(header='true', \
                       inferschema='true').\
            load("data/wine_dataset.csv",header=True);

In [4]:
df.show(5)
df.printSchema()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|style|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|  red|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|  red|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|  red|
|         11.2|            0.28|       0.56|           1.9|    0

In [5]:
labelCol='style'

In [6]:
catcols=[]
for i in df.dtypes:
    if i[1] == 'string' and i[0] != labelCol:
        catcols.append(i[0])

In [7]:
num_cols=[]
for i in df.dtypes:
    if i[1] != 'string' and i[0] != labelCol:
        num_cols.append(i[0])

In [8]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# categorical columns
categorical_columns = catcols

In [9]:
indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c)) for c in categorical_columns ]

In [10]:
encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(), \
                           outputCol="{0}_encoded".format(indexer.getOutputCol())) for indexer in indexers ]

In [11]:
assembler = VectorAssembler(inputCols=[encoder.getOutputCol() \
                                       for encoder in encoders] + num_cols, outputCol="features")

In [12]:
pipeline = Pipeline(stages=indexers + encoders + [assembler])
model=pipeline.fit(df)
data = model.transform(df)
data = data.withColumn('label',col(labelCol))
data=data.select('features','label')
data.show(5, truncate=False)

+------------------------------------------------------------+-----+
|features                                                    |label|
+------------------------------------------------------------+-----+
|[7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0]  |red  |
|[7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5.0]  |red  |
|[7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5.0] |red  |
|[11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,6.0]|red  |
|[7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0]  |red  |
+------------------------------------------------------------+-----+
only showing top 5 rows



In [13]:
def findTopPrincipalComponentNumberSVD(data:pyspark.sql.dataframe.DataFrame=df, \
                                    featureCol:str='features',threshold:int=0.80) -> (int, float):
    df_detect=data.select(featureCol)
    df_detect_list=[]
    for i in df_detect.collect():
        df_detect_list.append(i[0].tolist())
    df_detect_np=np.array(df_detect_list)
    dfDotDfTranspose=np.matmul(df_detect_np, df_detect_np.transpose())
    eigen_vals, eigen_vecs = np.linalg.eig(dfDotDfTranspose) #Get list of Eigenvalues and list of Eigenvectors, 
                                                             #numbef of Eigenvalues = number of the feature columns 
    tot = sum(eigen_vals)
    var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)] #Sort the Eigenvalues in descending order
                                                                    #get the percentage of each Eigenvalue against 
                                                                    #sum of total eigenvalues
                                                                    #Then find number cumulative percentage from
                                                                    #first few largest eigenvalues to see threshold 
                                                                    #is crossed.  If so, you only need these few 
                                                                    #number of columns.
    cumulativePercept=0
    cumulativePerceptList=[]
    for i in var_exp:
        cumulativePercept+=i
        cumulativePerceptList.append(cumulativePercept)
        if cumulativePercept >= threshold:
            break
    return (len(cumulativePerceptList), float(cumulativePerceptList[-1]))

In [ ]:
bestK, coveredPercentage = findTopPrincipalComponentNumberSVD(data=data, featureCol='features', threshold=0.99)

In [ ]:
bestK

In [ ]:
coveredPercentage

In [ ]:
from pyspark import SparkContext
# $example on$
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

In [ ]:
sc=spark.sparkContext

In [ ]:
rows=data.select("features").rdd.map(lambda r: [Vectors.dense(r[:])])

In [ ]:
mat = RowMatrix(rows)

In [ ]:
svd = mat.computeSVD(bestK, computeU=True)

In [ ]:
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.
# $example off$
collected = U.rows.collect()
print("U factor is:")
for vector in collected:
    print(vector)
    print("Singular values are: %s" % s)
    print("V factor is:\n%s" % V)

In [ ]:
uList=[]
for i in U.rows.collect():
    uList.append(i[:])
uListNP=np.array(uList)

In [ ]:
uListNP.shape

In [ ]:
S=np.diag(s.toArray())

In [ ]:
S.shape

In [ ]:
US=np.matmul(uListNP,S)

In [ ]:
US.shape #This is reduced dimension dataset

In [ ]:
V.toArray().transpose().shape

In [ ]:
#You do not need to multiply with V transpose
#Unless you want to project it back to original nxm space
# USV=np.matmul(US, V.toArray().transpose()) 

In [ ]:
#USV.shape

In [ ]:
import pandas as pd
df_svd=spark.createDataFrame(pd.DataFrame(US))


In [ ]:
df_svd.show(3)

In [ ]:
df_svd.count()

In [ ]:
assembler = VectorAssembler(inputCols=df_svd.columns, outputCol="features")

In [ ]:
df_svd=assembler.transform(df_svd).select("features")
df_svd.show(3, truncate=False)

In [ ]:
df_label=data.select("label")

In [ ]:
df_label.count()

In [ ]:
svdList=[]
for i in df_svd.collect():
    svdList.append(i[0])
labelList=[]
for i in df_label.collect():
    labelList.append(i[0])
combo=[(i[0],i[1]) for i in zip(svdList, labelList)]

In [ ]:
spark.createDataFrame(combo, ["features","label"]).show()

In [ ]:
#!pip install scikit-image

In [ ]:
from skimage.color import rgb2gray
from skimage import data
import matplotlib.pyplot as plt

In [ ]:
cat = data.chelsea()
plt.imshow(cat)
# convert to grayscale
gray_cat = rgb2gray(cat)

In [ ]:
plt.imshow(gray_cat, cmap="gray")

In [ ]:
import pandas as pd
df_cat=spark.createDataFrame(pd.DataFrame(gray_cat))

In [ ]:
from pyspark import SparkContext
# $example on$
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

In [ ]:
sc=spark.sparkContext

In [ ]:
rows=df_cat.rdd.map(lambda r: [Vectors.dense(r[:])])

In [ ]:
mat = RowMatrix(rows)

In [ ]:
mat

In [ ]:
cols=df_cat.columns
assembler = VectorAssembler(inputCols=cols, outputCol="features")
data=assembler.transform(df_cat)
bestK, coveredPercentage = findTopPrincipalComponentNumberSVD(data=data, featureCol='features', threshold=0.99)

In [ ]:
bestK

In [ ]:
svd = mat.computeSVD(bestK, computeU=True)

In [ ]:
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.
# $example off$
collected = U.rows.collect()
print("U factor is:")
for vector in collected:
    print(vector)
    print("Singular values are: %s" % s)
    print("V factor is:\n%s" % V)

In [ ]:
uList=[]
for i in U.rows.collect():
    uList.append(i[:])

In [ ]:
uListNP=np.array(uList)

In [ ]:
uListNP[0:3]

In [ ]:
s

In [ ]:
S=np.diag(s.array)

In [ ]:
S

In [ ]:
US=np.matmul(uListNP,S)

In [ ]:
US.shape

In [ ]:
USV=np.matmul(US, V.toArray().transpose())

In [ ]:
USV.shape

In [ ]:
plt.imshow(USV, cmap="gray")

In [ ]:
plt.imshow(gray_cat, cmap="gray") #Original picture

In [ ]:
compressedSize=uListNP.size+s.size+V.toArray().size

In [ ]:
originalSize=gray_cat.size

In [ ]:
compressionRate=compressedSize/originalSize*100

In [ ]:
compressionRate

In [ ]:
rank=min(USV.shape[0], bestK)

In [ ]:
rank

In [ ]:
coveredPercentage